In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!unzip "/kaggle/input/denoising-dirty-documents/train.zip" -d train_dataset
!unzip "/kaggle/input/denoising-dirty-documents/train_cleaned.zip" -d train_cleaned_dataset
!unzip "/kaggle/input/denoising-dirty-documents/test.zip" -d test_dataset


Archive:  /kaggle/input/denoising-dirty-documents/train.zip
   creating: train_dataset/train/
  inflating: train_dataset/train/101.png  
  inflating: train_dataset/train/102.png  
  inflating: train_dataset/train/104.png  
  inflating: train_dataset/train/105.png  
 extracting: train_dataset/train/107.png  
 extracting: train_dataset/train/108.png  
 extracting: train_dataset/train/11.png  
  inflating: train_dataset/train/110.png  
 extracting: train_dataset/train/111.png  
  inflating: train_dataset/train/113.png  
  inflating: train_dataset/train/114.png  
 extracting: train_dataset/train/116.png  
  inflating: train_dataset/train/117.png  
 extracting: train_dataset/train/119.png  
 extracting: train_dataset/train/12.png  
 extracting: train_dataset/train/120.png  
  inflating: train_dataset/train/122.png  
 extracting: train_dataset/train/123.png  
  inflating: train_dataset/train/125.png  
  inflating: train_dataset/train/126.png  
  inflating: train_dataset/train/128.png  
  inf

In [3]:
# torch vision and framework used for image preprocess
import torch
from PIL import Image
from torchvision import transforms
import os

In [4]:

TRAIN_IMAGE_PATH = '/kaggle/working/train_cleaned_dataset/train_cleaned/'

for i in range(101, 110):
    img_path = os.path.join(TRAIN_IMAGE_PATH, f"{i}.png")
    
    try:
        img = Image.open(img_path)
        width, height = img.size
        print(f"{i}.png -> Width: {width}px, Height: {height}px")
    except FileNotFoundError:
        print(f"{i}.png not found.")
    except UnidentifiedImageError:
        print(f"{i}.png could not be opened (maybe corrupted or unsupported).")
# size of image will be 540 x 420

101.png -> Width: 540px, Height: 420px
102.png -> Width: 540px, Height: 420px
103.png not found.
104.png -> Width: 540px, Height: 420px
105.png -> Width: 540px, Height: 420px
106.png not found.
107.png -> Width: 540px, Height: 420px
108.png -> Width: 540px, Height: 420px
109.png not found.


In [12]:
IMAGE_DIRTY_PATH = '/kaggle/working/train_dataset/train/'
for i in range(101, 110):
    img_path = os.path.join(TRAIN_IMAGE_PATH, f"{i}.png")
    try:
        img = Image.open(img_path)
        width, height = img.size
        print(f"{i}.png -> Width: {width}px, Height: {height}px")
    except FileNotFoundError:
        print(f"{i}.png not found.")
        

# original will be 540 x 420
# convert it to tensor

101.png -> Width: 540px, Height: 420px
102.png -> Width: 540px, Height: 420px
103.png not found.
104.png -> Width: 540px, Height: 420px
105.png -> Width: 540px, Height: 420px
106.png not found.
107.png -> Width: 540px, Height: 420px
108.png -> Width: 540px, Height: 420px
109.png not found.


In [13]:
# get the image for training 
train_image = []
evaluated_image = []
list_name = []
# convert images into tensor into the tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

for filename in os.listdir(TRAIN_IMAGE_PATH):
    if filename.lower().endswith('.png'):
        list_name.append(filename)
        img_path = os.path.join(TRAIN_IMAGE_PATH, filename)
        img = Image.open(img_path)
        img = img.convert('RGB')
        img_tensor = transform(img)
        evaluated_image.append(img_tensor)

for filename in list_name:
    img_path = IMAGE_DIRTY_PATH + filename
    ìmg = Image.open(img_path)
    img = img.convert('RGB')
    img_tensor = transform(img)
    train_image.append(img_tensor)

In [9]:
len(train_image)

144

In [14]:
len(evaluated_image)

144

In [15]:
train_image[0].shape

torch.Size([3, 258, 540])

In [27]:
evaluated_image

[tensor([[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.1804, 2.1975, 2.2147],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 0.1939, 0.3994, 1.1015]],
 
         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.3585, 2.3761, 2.3936],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 0.3277, 0.5378, 1.2556]],
 
         [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ...,

In [16]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [28]:
class ConvolutionalAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvolutionalAutoEncoder, self).__init__()

        # encoder part
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, stride=2, padding=1), #[3, 258, 540] -> [15, 129, 270]
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1), # [32, 65, 135]
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels= 64, kernel_size=3, stride=2, padding = 1), # [64, 33, 68]
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding = 1, output_padding=(1,1)),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16,3, stride=2, padding= 1, output_padding=(1,1)),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding = (1, 0)),
            # nn.Sigmoid() # for generate the grey type 
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [29]:
model = ConvolutionalAutoEncoder()
x = torch.randn(1, 3, 258, 540)  # batch size 1
out = model(x)
print(out.shape)  # torch.Size([1, 3, 258, 540])


torch.Size([1, 3, 264, 543])


In [30]:
out

tensor([[[[ 0.1175,  0.1478,  0.1143,  ...,  0.1181,  0.1482,  0.1158],
          [ 0.1504,  0.1638,  0.1614,  ...,  0.1478,  0.1632,  0.1569],
          [ 0.1146,  0.1456,  0.1041,  ...,  0.1156,  0.1436,  0.1083],
          ...,
          [ 0.1479,  0.1644,  0.1543,  ...,  0.1490,  0.1652,  0.1541],
          [ 0.1138,  0.1452,  0.1102,  ...,  0.1133,  0.1472,  0.1127],
          [ 0.1397,  0.1352,  0.1327,  ...,  0.1394,  0.1409,  0.1335]],

         [[-0.0861, -0.0501, -0.0865,  ..., -0.0859, -0.0550, -0.0860],
          [-0.0842, -0.1002, -0.0838,  ..., -0.0856, -0.0906, -0.0843],
          [-0.0828, -0.0453, -0.0914,  ..., -0.0852, -0.0448, -0.0900],
          ...,
          [-0.0862, -0.0924, -0.0776,  ..., -0.0844, -0.0898, -0.0805],
          [-0.0831, -0.0465, -0.0909,  ..., -0.0845, -0.0509, -0.0888],
          [-0.0900, -0.0571, -0.0910,  ..., -0.0919, -0.0557, -0.0865]],

         [[-0.0878, -0.0560, -0.0882,  ..., -0.0884, -0.0519, -0.0867],
          [-0.0551, -0.0982, -

In [31]:
x

tensor([[[[-0.4784,  0.0961,  1.8321,  ..., -2.1330, -0.4216,  0.9448],
          [-0.4141,  0.4998,  0.0605,  ..., -0.7753, -1.2131, -0.7184],
          [-0.1482,  0.6134, -0.8711,  ...,  0.2533,  0.6672, -1.4202],
          ...,
          [ 0.0098,  1.4565,  0.0109,  ...,  0.8675, -0.7901, -1.2409],
          [-1.4519,  0.8427, -0.8626,  ...,  1.5742, -0.9994, -0.7057],
          [-0.7200,  0.8732,  0.2424,  ...,  0.4215,  1.1039, -0.5002]],

         [[-1.1951, -0.7391, -1.0359,  ..., -1.3780, -1.3006, -0.6955],
          [ 0.6914, -0.0392, -0.8687,  ..., -0.0388, -1.7437, -0.3688],
          [ 1.4054,  0.4315, -1.1015,  ...,  0.5308,  0.0160, -0.8542],
          ...,
          [ 0.4716,  1.2318, -0.1678,  ..., -1.1295, -0.9995,  0.7455],
          [-1.0095, -1.1666,  0.8215,  ...,  0.0580, -0.9224,  2.7648],
          [ 0.0059,  0.3902,  1.4805,  ..., -0.8105, -1.6546,  1.2799]],

         [[ 0.2987,  0.0518,  0.2488,  ..., -1.0800,  0.1396, -1.3789],
          [ 0.9444, -0.7452, -

In [33]:
from torch.utils.data import Dataset, DataLoader


In [89]:
class DenoisedDataset(Dataset):
    def __init__(self, train_x, train_y):
        super().__init__()
        self.train_x = train_x
        self.train_y = train_y
    def __len__(self):
        return len(self.train_x)
    def __getitem__(self, idx):
        return torch.tensor(self.train_x[idx]), torch.tensor(self.train_y[idx])

In [90]:
train_idx = int(0.8 * len(train_image))

In [91]:
train_dataset = DenoisedDataset(train_image[:train_idx], evaluated_image[:train_idx])
val_dataset = DenoisedDataset(train_image[train_idx:], evaluated_image[train_idx:])

In [92]:
train_dataset

In [93]:
val_dataset

In [94]:
train_dataset.__len__()

115

In [95]:
val_dataset.__len__()

29

In [96]:
train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size = 8, shuffle=True)

In [97]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [98]:
device

'cpu'

In [99]:
def evaluate(model, val_dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for train_x, train_y in val_dataloader:
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            pred_y = model(train_x)
            loss = criterion(pred_y, y)
            total_loss += loss.item()
    return total_loss / len(val_dataloader)

In [100]:
from torch import optim
def train(model, train_dataloader, epochs, lr, device):
    criterion = nn.MSELoss()
    params = list(model.parameters())
    num_trainable_params = sum(p.numel() for p in params if p.requires_grad)
    print(f"Total number of trainable parameters: {num_trainable_params}")
    optimizer = optim.Adam(params, lr=lr)
    model.to(device)
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch_idx, (train_x, train_y) in enumerate(train_dataloader):
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            optimizer.zero_grad()
            pred_y = model(train_x)
            loss = criterion(pred_y, train_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            print(f'Epoch {epoch + 1}/{epochs}, Step {batch_idx + 1}/ {len(train_dataloader)}, Loss = {loss.item()}')
        val_loss = evaluate(model, val_dataloader, criterion, device)
        print(f"Epoch [{epoch+1}/{epochs}] Validation Loss: {val_loss:.4f}")

        # Print average training loss
        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch [{epoch+1}/{epochs}] Average Training Loss: {avg_loss:.4f}")

In [101]:
model = ConvolutionalAutoEncoder()
train(model, train_dataloader, 10, 1e-4, device)

Total number of trainable parameters: 47107


/tmp/ipykernel_31/1745399332.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.train_x[idx]), torch.tensor(self.train_y[idx])


RuntimeError: stack expects each tensor to be equal size, but got [3, 420, 540] at entry 0 and [3, 258, 540] at entry 3